# Match Meise Collectors to Wikidata Items Using *Nearest Neighbour*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive.

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:

- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists.ipynb](./create_wikidata_datasets_botanists.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os
wikidata = pd.read_csv("data/wikidata_persons_botanists_20230703_1352.csv", 
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }
)

wikidata.head()

,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,...,bionomia_id,yob,yod,wyb,wye,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q66612,Friedrich August Marschall von Bieberstein,Bieberstein,F.A.M.v.,"Bieberstein, F.A.M.v.","Bieberstein, Friedrich August Marschall von",NaN,43340073,0000 0001 1630 5464,1373,...,Q66612,1768,1826,<NA>,<NA>,http://www.wikidata.org/wiki/Q66612,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/6129-1,https://bionomia.net/Q66612
1,http://www.wikidata.org/entity/Q66934,Hans Hermann Behr,Behr,H.H.,"Behr, H.H.","Behr, Hans Hermann",NaN,20328622,0000 0001 1604 8680,42741,...,Q66934,1818,1904,<NA>,<NA>,http://www.wikidata.org/wiki/Q66934,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/619-1,https://bionomia.net/Q66934
2,http://www.wikidata.org/entity/Q66661,Jacob Christian Schäffer,Schäffer,J.C.,"Schäffer, J.C.","Schäffer, Jacob Christian",NaN,47016953,0000 0000 8343 3899,1101,...,NaN,1718,1790,<NA>,<NA>,http://www.wikidata.org/wiki/Q66661,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/12818-1,NaN
3,http://www.wikidata.org/entity/Q67003,Johann Friedrich Klotzsch,Klotzsch,J.F.,"Klotzsch, J.F.","Klotzsch, Johann Friedrich",NaN,20426762,0000 0001 1749 2732,135,...,Q67003,1805,1860,<NA>,<NA>,http://www.wikidata.org/wiki/Q67003,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/4855-1,https://bionomia.net/Q67003
4,http://www.wikidata.org/entity/Q66322,Franz Anton Menge,Menge,F.A.,"Menge, F.A.","Menge, Franz Anton",NaN,59847236,0000 0001 1653 0899,73782,...,NaN,1808,1880,<NA>,<NA>,http://www.wikidata.org/wiki/Q66322,NaN,https://kiki.huh.harvard.edu/databases/botanis...,https://www.ipni.org/a/23266-1,NaN


In [2]:
# Create data frame with unique canonical strings 
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

display(wd_matchtest)
display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"(-Walraevens), O.H.",1
1,"(1835-1906), G.A.F.E.",1
2,"(1873-1926), S.S.",1
3,"(1888–1973), G.A.",1
4,"(1904-1990), J.J.",1
...,...,...
61479,"Șerbanescu, I.",1
61480,"Ștefureac, T.",1
61481,"Țopa, E.",1


,canonical_string_fullname,item
,,count
0,"(-Walraevens), O Heylen",1
1,"(1835-1906), Gustav Adolf Ferdinand Eichler",1
2,"(1873-1926), Søren Sørensen",1
3,"(1888–1973), Georges André",1
4,"(1904-1990), Johannes Johannessen",1
...,...,...
63605,"Șerbanescu, Ioan",1
63606,"Ștefureac, Traian",1
63607,"Țopa, Emilian",1


### Load Collectors Data Set

Data sources:

- Jupyter Notebook for [create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_meise_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`


In [3]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/meise_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv(
    "./data/Meise_doi-10.15468-dl.ax9zkh/occurrence_recordedBy_eventDate_occurrenceIDs_20230830_parsed.tsv", 
    sep="\t", low_memory=False
)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

# Out of bounds nanosecond timestamp: 1652-01-01T00:00:00
#  because date nanoseconds range limitations of pandas, see https://stackoverflow.com/a/69507200/1240387
print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='S' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
collectors

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
43575,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,H.M. L[lo][???]a,parsed:H.M.Llo a,cleaned:H. M. Llo A,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
84443,A,R.,NaN,NaN,NaN,NaN,NaN,NaN,R. Decha[] & R. []a,parsed:R. Decha<SEP>R. a,cleaned:R. Decha<SEP>R. A,1,http://www.botanicalcollections.be/specimen/BR...,1988-11-19 00:00:00,1988-11-19,1988-11-19
94546,A,Soc Merc,NaN,NaN,NaN,NaN,NaN,NaN,Soc. Merc. A.[?F.],parsed:Soc Merc A,cleaned:Soc Merc A,1,http://www.botanicalcollections.be/specimen/BR...,1841-01-01 00:00:00,1841-01-01,1841-01-01
18402,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Church, A.C., Ismail, Ruskandi, A",parsed:A.C. Church<SEP>Ruskandi Ismail<SEP>A,cleaned:A.C. Church<SEP>Ruskandi Ismail<SEP>A,1,http://www.botanicalcollections.be/specimen/BR...,1995-10-26 00:00:00,1995-10-26,1995-10-26
4693,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.[...],parsed:A,cleaned:A,1,http://www.botanicalcollections.be/specimen/BR...,1896-09-01 00:00:00,1896-09-01,1896-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24908,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,Dr. A. Žertová,parsed:A. Žertová,cleaned:A. Žertová,6,http://www.botanicalcollections.be/specimen/BR...,1959-01-01 04:00:00,1958-08-13,1960-06-23
58887,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Klášterský & Žertová,parsed:Klášterský<SEP>Žertová,cleaned:Klášterský<SEP>Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20
118317,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ƶ,parsed:Ƶ,cleaned:ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00,1850-07-09,1850-07-09
91600,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",parsed:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. Ho...,cleaned:S.P. Pinnerup<SEP>B. Ǿllgaard<SEP>L. H...,1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29


#### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
criterion_fullnames = collectors.given.str.contains('^\w{3,}', na=False)
print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
collectors[criterion_fullnames]

Show collecors given name has (propably) a full name (17943 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
94546,A,Soc Merc,NaN,NaN,NaN,NaN,NaN,NaN,Soc. Merc. A.[?F.],parsed:Soc Merc A,cleaned:Soc Merc A,1,http://www.botanicalcollections.be/specimen/BR...,1841-01-01 00:00:00,1841-01-01,1841-01-01
64247,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,Lochenies in A. Brabant,parsed:Lochenies in A. Brabant,cleaned:Lochenies in A. Brabant,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00,1892-01-01,1892-01-01
34235,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,Fleury in A. Chevalier,parsed:Fleury in A. Chevalier,cleaned:Fleury in A. Chevalier,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
22243,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,Dandois in A. Cogniaux,parsed:Dandois in A. Cogniaux,cleaned:Dandois in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00,1866-06-01,1866-06-01
39848,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,Gilbert in A. Cogniaux,parsed:Gilbert in A. Cogniaux,cleaned:Gilbert in A. Cogniaux,2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00,1864-08-06,1864-08-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95120,Čermak,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan. Čermak,parsed:Stan Čermak,cleaned:Stan Čermak,1,http://www.botanicalcollections.be/specimen/BR...,1926-05-01 00:00:00,1926-05-01,1926-05-01
95115,Čermák,Stan,NaN,NaN,NaN,NaN,NaN,NaN,Stan Čermák,parsed:Stan Čermák,cleaned:Stan Čermák,1,http://www.botanicalcollections.be/specimen/BR...,1925-10-12 00:00:00,1925-10-12,1925-10-12
97327,Şes-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şes[]-Frăţilescu,parsed:Tatiana Şes-Frăţilescu,cleaned:Tatiana Şes-Frăţilescu,1,http://www.botanicalcollections.be/specimen/BR...,1972-06-13 00:00:00,1972-06-13,1972-06-13
97328,Şesan-Frăţilescu,Tatiana,NaN,NaN,NaN,NaN,NaN,NaN,Tatiana Şesan-Frăţilescu,parsed:Tatiana Şesan-Frăţilescu,cleaned:Tatiana Şesan-Frăţilescu,9,http://www.botanicalcollections.be/specimen/BR...,1972-04-06 08:00:00,1971-07-12,1973-06-12


In [5]:
# check the parsed columns if they are empty or need to be considerd as data for matching or not
for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
    test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
    print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
    display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 8189 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
12639,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN
64247,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN
34235,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN
49411,A. Chrková,J. Chrtek,NaN,at,NaN,NaN,NaN,NaN
22243,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 191 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
35043,Almeda,Frank,Jr.,NaN,NaN,NaN,NaN,NaN
69099,Anderson,W.A.,Jr.,NaN,NaN,NaN,NaN,NaN
55984,Atham,John,Jr,NaN,NaN,NaN,NaN,NaN
52811,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN
61589,Bailey,L.H.,Jr.,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1030 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
34741,A. Hardy,Lebrun,NaN,in,NaN,NaN,Fr.,NaN
117371,Abbon,G.,NaN,NaN,NaN,NaN,fr.,NaN
108996,Ahlberg,NaN,NaN,NaN,NaN,NaN,Fr.,NaN
34686,Ahlfvengren,E.,NaN,NaN,NaN,NaN,Fr.,NaN
117892,Alavoin,NaN,NaN,NaN,NaN,NaN,mr.,NaN


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [6]:
collectors['canonical_string_collector_parsed'] = (
  # use collectors.family only where given name has NA values, otherwise use family name + given name
  collectors.family.where(
      # condition
      collectors.given.isna(),
      # any other 
      # TODO improve the combined name for canonical_string_collector_parsed if any of the other dwc_parsed fields is not NaN
      other= (collectors.family + ", " + collectors.given) \
        if any(collectors.particle.isna()) \
        else collectors.particle + " " + collectors.family + ", " + collectors.given
  )
)
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
24908,Žertová,A.,NaN,NaN,NaN,NaN,NaN,Dr.,"Žertová, A.",Dr. A. Žertová
58887,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová
118317,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,Ƶ
91600,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.","S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen"
118320,Т. Королева. V. Petrovsky,В.Пертовский,NaN,и,NaN,NaN,NaN,NaN,"Т. Королева. V. Petrovsky, В.Пертовский",В. Пертовский и Т. Королева. V. Petrovsky & T....


In [7]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)

display(collectors_unique)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",32,http://www.botanicalcollections.be/specimen/BR...,1905-09-12 10:40:00,1843-09-02,1995-10-26
1,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",Br. [Dedothaures] in A Robyns,1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00,1954-08-01,1954-08-01
2,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00,1864-05-01,1864-05-01
3,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,"A, H. M. Llo",H.M. L[lo][???]a,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT
4,A,R.,NaN,NaN,NaN,NaN,NaN,NaN,"A, R.",R. Decha[] & R. []a,1,http://www.botanicalcollections.be/specimen/BR...,1988-11-19 00:00:00,1988-11-19,1988-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67467,Žertová,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Žertová,Klášterský & Žertová,1,http://www.botanicalcollections.be/specimen/BR...,1958-05-20 00:00:00,1958-05-20,1958-05-20
67468,Žertová,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Žertová, A.",A. Žertová,20,http://www.botanicalcollections.be/specimen/BR...,1959-04-19 20:00:00,1958-07-09,1963-06-27
67469,ƶ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ƶ,Ƶ,1,http://www.botanicalcollections.be/specimen/BR...,1850-07-09 00:00:00,1850-07-09,1850-07-09
67470,Ǿllgaard,B.,NaN,NaN,NaN,NaN,NaN,NaN,"Ǿllgaard, B.","S.P. Pinnerup, B. Ǿllgaard & L. Holm-Nielsen",1,http://www.botanicalcollections.be/specimen/BR...,1975-08-29 00:00:00,1975-08-29,1975-08-29


In [8]:
# show collectors with highest occurrenceID_collectors_count
collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family'], ascending=[False, True]).head(10)

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
62653,Vanderyst,H.,NaN,NaN,NaN,NaN,NaN,NaN,"Vanderyst, H.",Vanderyst H. & Polis C.,41259,http://www.botanicalcollections.be/specimen/BR...,1914-09-09 04:44:15,1819-06-07,1990-04-01
2121,Aptroot,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Aptroot, A.",Aptroot A.,32921,http://www.botanicalcollections.be/specimen/BR...,1989-07-24 07:01:45,1874-04-03,2022-08-03
38019,Louis,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Louis, J.",[J.Louis?],30727,http://www.botanicalcollections.be/specimen/BR...,1938-05-08 00:55:43,1875-08-01,1987-08-29
62573,Vanden Berghen,C.,NaN,NaN,NaN,NaN,NaN,NaN,"Vanden Berghen, C.",Duvigneaud P. & Vanden Berghen C.,29241,http://www.botanicalcollections.be/specimen/BR...,1967-06-16 14:58:50,1856-04-03,2000-01-20
39491,Malaisse,F.,NaN,NaN,NaN,NaN,NaN,NaN,"Malaisse, F.",Malaisse F. & Saad L.,26240,http://www.botanicalcollections.be/specimen/BR...,1996-04-08 05:10:00,1802-01-01,2017-01-09
16459,Duvigneaud,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Duvigneaud, J.","P. Auquier, W. Belotte & J. Duvigneaud",23751,http://www.botanicalcollections.be/specimen/BR...,1975-08-27 06:43:43,1874-06-26,2021-09-15
12255,Coppejans,E.,NaN,NaN,NaN,NaN,NaN,NaN,"Coppejans, E.",Coppejans E.,23015,http://www.botanicalcollections.be/specimen/BR...,1995-07-08 23:20:34,1967-01-01,2014-03-13
14454,Delvosalle,L.,NaN,NaN,NaN,NaN,NaN,NaN,"Delvosalle, L.",Delvosalle L.,22529,http://www.botanicalcollections.be/specimen/BR...,1963-10-15 04:43:01,1900-06-24,2013-06-01
59713,Symoens,J.,NaN,NaN,NaN,NaN,NaN,NaN,"Symoens, J.",Hoffmann E. & Symoens J.,22073,http://www.botanicalcollections.be/specimen/BR...,1966-05-16 18:50:07,1848-06-01,2017-01-09
2449,Arts,T.,NaN,NaN,NaN,NaN,NaN,NaN,"Arts, T.",Arts T.,21420,http://www.botanicalcollections.be/specimen/BR...,1989-04-12 02:03:50,1888-03-15,2000-11-26


In [9]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [11]:
import re
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title()  # normalise case - capital at start of each word
    string = re.sub(' +', ' ', string).strip() # get rid of multiple spaces and replace with a single
    string = ' ' + string + ' '  # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD', r'', string)
    this_ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in this_ngrams]

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


In [12]:
print("Show ngram examples:")
print("- simple name:", ngrams('Klazenga, N.'))
print("- data from collectors:", ngrams(collectors_unique["canonical_string_collector_parsed"].at[1])) 
print("- data from match-test:", ngrams(wd_matchtest['canonical_string'].at[0]))
print("- data from match-test (full name):", ngrams(wd_matchtest_fullnames['canonical_string_fullname'].at[3]))

Show ngram examples:
- simple name: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- data from collectors: [' A ', 'A R', ' Ro', 'Rob', 'oby', 'byn', 'yns', 'ns ', 's B', ' Br', 'Br ']
- data from match-test: [' Wa', 'Wal', 'alr', 'lra', 'rae', 'aev', 'eve', 'ven', 'ens', 'ns ', 's O', ' Oh', 'Oh ']
- data from match-test (full name): [' 18', '188', '888', '881', '819', '197', '973', '73 ', '3 G', ' Ge', 'Geo', 'eor', 'org', 'rge', 'ges', 'es ', 's A', ' An', 'And', 'ndr', 'dr ']


In [10]:
# some example data
for i, row in enumerate(range(5)):
    if (i == 0):
        print('(WikiData’s) canonical_string = (constructed) canonical_string_fullname') 
    pprint.pprint("%s = %s" % (
        wd_matchtest['canonical_string'].at[row],
        wd_matchtest_fullnames['canonical_string_fullname'].at[row]
    ))

(WikiData’s) canonical_string = (constructed) canonical_string_fullname
'(-Walraevens), O.H. = (-Walraevens), O Heylen'
'(1835-1906), G.A.F.E. = (1835-1906), Gustav Adolf Ferdinand Eichler'
'(1873-1926), S.S. = (1873-1926), Søren Sørensen'
'(1888–1973), G.A. = (1888–1973), Georges André'
'(1904-1990), J.J. = (1904-1990), Johannes Johannessen'


Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (Meise) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [13]:
# pip install --upgrade scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# nbrs_data = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf_vector_data) 
# tfidf_vector_data contains the vectorized wikidata names from the previous step


def getNearestNeighbour(query, this_vectorizer, this_nbrs_data):
    """Calculate the k-nearest distance for query data using package scikit-learn


    @param query: DataFrame the query data to vectorize and transform
    @param this_vectorizer: the vectorizer of TfidfVectorizer
    @param this_nbrs_data: the data of NearestNeighbors calculations
    @return: (distances, indices) distances and indices
    @rtype (int, int)
    """
    queryTFIDF_ = this_vectorizer.transform(query)
    distances, indices = this_nbrs_data.kneighbors(queryTFIDF_)
    return distances, indices


def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: Number of neighbors required for each sample by default for :meth:`kneighbors` queries (originally 5).

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """

    import time
    start = time.time()
    query_data = set(query_data)
    # convert list to set for better performance

    print('Vectorizing data. This may take a while...')
    # vectorize wikidata names
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_vector_data = vectorizer.fit_transform(match_data
        # wd_matchtest['canonical_string']
    )
    nbrs_data = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1).fit(tfidf_vector_data)
    duration = time.time() - start
    print('Vectorizing completed: created a matrix of TF-IDF features after %s s' % duration)

    print('Getting nearest neighbours of %s data with %s neighbor sample(s)...' % (len(query_data), n_neighbors))
    distances, indices = getNearestNeighbour(query_data, vectorizer, nbrs_data)
    duration = time.time() - start
    print('Completed after %s s' % duration)

    query_data = list(query_data)  # convert back to list

    print('Finding matches build new data frame ...')
    matches = []
    for i, j in enumerate(indices):
        temp = [query_data[i], match_data.values[j][0], round(distances[i][0], 2)]
        matches.append(temp)

    duration = time.time() - start
    print('Building matches done after %s s' % duration)
    matches = pd.DataFrame(
        matches,
        columns=['namematch_source_data', 'namematch_resource_data', 'namematch_distance']
    )

    print('Done')
    return matches


In [14]:
criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
print("Calculate matching for **abbrevated** names separately …")
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

display(matches)

Calculate matching for **abbrevated** names separately …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 2.7878825664520264 s
Getting nearest neighbours of 54382 data with 5 neighbor sample(s)...
Completed after 302.7156620025635 s
Finding matches build new data frame ...
Building matches done after 303.417293548584 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,27190,"Gottschlich, G.","Gottschlich, G.",0.00
1,41144,"Johnston, M.C.","Johnston, M.C.",0.00
2,11244,"Adhikari, M.K.","Adhikari, M.K.",0.00
3,41146,"Sbarbaro, C.","Sbarbaro, C.",0.00
4,26267,"Fukuoka, N.","Fukuoka, N.",0.00
...,...,...,...,...
54377,31126,Onzichtbaar,"Saar, M.",1.25
54378,4908,"Sduihuyk, W.","Mack, W.",1.25
54379,19507,"Oluadnunle, A.","Plée, A.",1.25
54380,649,"Cercle Scientifique, H.Lücher","Manrique, H.F.",1.25


In [15]:
# criterion_fullnames = collectors_unique.given.str.contains('^\w{3,}', na=False)
print("Calculate matching for **full** names separately …")
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'], 5) # TODO what effect has n_neighbors ? originally in the very source code it is set to 5, not 1

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

display(matches_fullnames)


Calculate matching for **full** names separately …
Vectorizing data. This may take a while...
Vectorizing completed: created a matrix of TF-IDF features after 3.3555400371551514 s
Getting nearest neighbours of 13090 data with 5 neighbor sample(s)...
Completed after 176.8270080089569 s
Finding matches build new data frame ...
Building matches done after 176.93333387374878 s
Done


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,9023,"Madani, Leopold","Madani, Leopold",0.00
1,8782,"Turk, Jean","Turk, Jean",0.00
2,8784,"McPherson, Gordon D.","McPherson, Gordon D",0.00
3,8823,"Bonar, Lee","Bonar, Lee",0.00
4,8837,"Koch, Leo","Koch, Leo",0.00
...,...,...,...,...
13085,7126,"Wellemeersen, Begeerswerkgroep","Hansen, Steen Gregersen",1.26
13086,475,"Looz, Ajumpfuho","Strumpf, Edward",1.26
13087,4910,"Sjine, Evu J.G.","Antoine, E",1.26
13088,10255,"ClJunojum, Neopresbyter","Catesby, Mark",1.27


### Create Output Results

Combine the matches data frame back to the (Meise) collectors and Wikidata items …

In [16]:
# join matches data frame back to source collectors  dataframe 
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,"Church, A.C., Ismail, Ruskandi, A",32,http://www.botanicalcollections.be/specimen/BR...,1905-09-12 10:40:00,1843-09-02,1995-10-26,17939,A,"Mas, A.",1.16
1,A Robyns,Br,NaN,in,NaN,NaN,NaN,NaN,"A Robyns, Br",Br. [Dedothaures] in A Robyns,1,http://www.botanicalcollections.be/specimen/BR...,1954-08-01 00:00:00,1954-08-01,1954-08-01,8734,"A Robyns, Br","Robyns, F.H.E.A.W.",0.92
2,A'hlbon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'hlbon,A'hlbon,1,http://www.botanicalcollections.be/specimen/BR...,1864-05-01 00:00:00,1864-05-01,1864-05-01,4457,A'hlbon,"Ahlburg, H.",1.01
3,A,H. M. Llo,NaN,NaN,NaN,NaN,NaN,NaN,"A, H. M. Llo",H.M. L[lo][???]a,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,43625,"A, H. M. Llo","Llorens, L.",1.13
4,A,R.,NaN,NaN,NaN,NaN,NaN,NaN,"A, R.",R. Decha[] & R. []a,1,http://www.botanicalcollections.be/specimen/BR...,1988-11-19 00:00:00,1988-11-19,1988-11-19,15226,"A, R.","Ma, R.",1.16


In [17]:
# append full name matches
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

collectors_matches_fullname.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,Soc Merc,NaN,NaN,NaN,NaN,NaN,NaN,"A, Soc Merc",Soc. Merc. A.[?F.],1,http://www.botanicalcollections.be/specimen/BR...,1841-01-01 00:00:00,1841-01-01,1841-01-01,7316,"A, Soc Merc","Bernal, Mercè",1.11
1,A. Brabant,Lochenies,NaN,in,NaN,NaN,NaN,NaN,"A. Brabant, Lochenies",Lochenies in A. Brabant,1,http://www.botanicalcollections.be/specimen/BR...,1892-01-01 00:00:00,1892-01-01,1892-01-01,5382,"A. Brabant, Lochenies","Lochenies, Georges",0.98
2,A. Chevalier,Fleury,NaN,in,NaN,NaN,NaN,NaN,"A. Chevalier, Fleury",Fleury in A. Chevalier,2,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,4025,"A. Chevalier, Fleury","Fleury, J F",0.97
3,A. Cogniaux,Dandois,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Dandois",Dandois in A. Cogniaux,1,http://www.botanicalcollections.be/specimen/BR...,1866-06-01 00:00:00,1866-06-01,1866-06-01,4445,"A. Cogniaux, Dandois","Cogniaux, Alfred",0.90
4,A. Cogniaux,Gilbert,NaN,in,NaN,NaN,NaN,NaN,"A. Cogniaux, Gilbert",Gilbert in A. Cogniaux,2,http://www.botanicalcollections.be/specimen/BR...,1864-08-06 00:00:00,1864-08-06,1864-08-06,2642,"A. Cogniaux, Gilbert","Cogniaux, Alfred",0.88


In [18]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
collectors_all_matches.head()

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
107,Aalto,M.,NaN,NaN,NaN,NaN,NaN,NaN,"Aalto, M.",M. Aalto,16,http://www.botanicalcollections.be/specimen/BR...,1976-07-05 11:12:00,1975-07-07,1976-09-09,29360,"Aalto, M.","Aalto, M.",0.0
114,Aas,O.,NaN,NaN,NaN,NaN,NaN,NaN,"Aas, O.",Aas O.,1,http://www.botanicalcollections.be/specimen/BR...,NaT,NaT,NaT,21588,"Aas, O.","Aas, O.",0.0
133,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbas, A.",A. Abbas,44,http://www.botanicalcollections.be/specimen/BR...,1934-10-15 01:39:19,1869-05-04,2001-08-10,42911,"Abbas, A.","Abbas, A.",0.0
137,Abbe,E.C.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbe, E.C.","H.M. Raup, E.C. Abbe",1,http://www.botanicalcollections.be/specimen/BR...,1932-09-01 00:00:00,1932-09-01,1932-09-01,39368,"Abbe, E.C.","Abbe, E.C.",0.0
144,Abbott,A.T.D.,NaN,NaN,NaN,NaN,NaN,NaN,"Abbott, A.T.D.","Abbott, A.T.D.",1,http://www.botanicalcollections.be/specimen/BR...,2010-01-07 00:00:00,2010-01-07,2010-01-07,27404,"Abbott, A.T.D.","Abbott, A.T.D.",0.0


In [19]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

### Merge and Aggregate Matched Data and WikiData’s

Review (TODO)
- evaluate time references: `eventDate` ~ `yob`, `wyb`—perhaps define a score value that could integrate all scores from properties we need for decision of the name matching (name distance, eventDate ~ year of birth/work year begin aso.)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables


In [20]:
# merge now the matching data and the wiki data’s on the conaonical string name
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

In [21]:
# # # # # # # # # # # # # # # # # # # # # # # # # 
# custom grouping, analysing and saving of data:
# # # # # # # # # # # # # # # # # # # # # # # # # 
# ## cell split - markdown
# # Save the plain name matching results only ...
# 
# ## cell split - code
# 
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20230913
# 
# this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
# 

In [22]:
# print("Show some name match examples (e.g. «Louis…» matching various names) …")
# for testname in ['Louis', 'Abbot']:
#     criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].map(lambda x: x.startswith(testname))    
#     this_table=collectors_matches_g1_merged_wikidata[criterion].get([
#         # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
#         'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
#         'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
#         # 'canonical_string_fullname', 
#         'itemLabel', 'wikidata_link',
#         'collectors_eventDate_min', 'collectors_eventDate_max',
#         'yob', 'yod', 'wyb', 'wye'
#     ]).sort_values(by=['namematch_distance'])
#     print("# ---------------------------------------------\n# «%s…» as test name, %d collector names begin with:" % (testname, criterion.sum()))    
#     display(this_table)
# 
# ## cell split - markdown
# # Aggregate data to get atomized listings of multiple resource name matches joining by “|” aso.
# ## cell split - code
# 
# print('Group data by canonical names (abbreviated and full name):'
#       ' multiple related WD items (e.g. Q1232456), item labels, year of birth, year of death')
# for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
#     print('Run %s:   Group by wiki data’s %s, and aggregate/join item(s), labels, yob, yod '
#           'by “…|…”, add new columns “…_joined” ...' % (i + 1, wd_matching_column))
#     wdata_joined_items_and_others = wikidata.groupby([wd_matching_column]).agg(
#         items_joined = ('item', lambda x: '|'.join(x)),
#         item_labels_joined = ('itemLabel', lambda x: '|'.join(x)),
#         yob_joined = ('yob', lambda x: '|'.join([str(s) for s in list(x)]) ),
#         yod_joined = ('yod', lambda x: '|'.join([str(s) for s in list(x)]) )
#     ).reset_index()
# 
#     # print("Done. Show examples of items having multiple matching data «|» … ")
#     # criterion = wdata_joined_items['items'].map(lambda x: '|' in x)
#     # wdata_joined_items[criterion].head()
# 
#     print('Run %s:   Merge all based on namematch_resource_data, add item(s) data ...' % (i + 1))
#     collectors_matches_g2 = pd.merge(
#         collectors_matches_g1_merged_wikidata, wdata_joined_items_and_others,
#         left_on='namematch_resource_data', right_on=wd_matching_column
#         , suffixes=('__wikidata_merge', '__grp_by_items')
#         # append to left-data, right-data only when identical column names occur
#     )
# 
#     print('Run %s:   Build data frame “collectors_matches_group” ...' % (i + 1))
#     collectors_matches_group = collectors_matches_g2 \
#         if i == 0 \
#         else pd.concat([collectors_matches_group, collectors_matches_g2], ignore_index = True)
#     
# print('Done')
# ## cell split - code
# 
# print("Show examples of item_labels_joined having multiple matching data «|» … ")
# criterion = collectors_matches_group['item_labels_joined'].map(lambda x: '|' in x)
# 
# collectors_matches_group[criterion].get([ # empty 
#     # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
#     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
#     'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
#     # 'canonical_string_fullname', 
#     'item_labels_joined', 'items_joined', 'yob_joined', 'yod_joined'
# ], default="...get: Are data empty or it has probably a wrong named column?")
# ## cell split - code
# 
# # check what columns we have and what we would keep for further analysis and what to drop
# pprint.pprint(collectors_matches_group.columns)
# # from merge: _x would means normally from left column, _y means from right column
# # in BASH fold text long lines; echo "${text}" | fold --spaces | sed 's@^@#  @'
# ## cell split - markdown
# 
# # Prepare data to save later on …
# ## cell split - code
# # Remove superfluous columns TODO check WARNING: A value is trying to be set on a copy of a slice from a DataFrame
# # TODO check duplicates
# collectors_matches_group_simplified = collectors_matches_group.get(
#     ['family', 'given', 'canonical_string_collector_parsed', 
#       'namematch_source_data', # redundant: 'namematch_source_data' == 'canonical_string_collector_parsed'
#       'namematch_resource_data', 'namematch_distance', 
#       'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max', # collecors dates
#       'yob_joined', 'yod_joined', # WikiData dates
#       'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
#       'items_joined', 'canonical_string', 'canonical_string_fullname', 'surname', 'initials', 'item_labels_joined'
#     ], default="...get: Are data empty or it has probably a wrong named column?"
# )
# # collectors_matches_group = collectors_matches_g3
# collectors_matches_group_simplified.sort_values(
#     by=['namematch_distance', 'canonical_string_collector_parsed']
#     , inplace=True
# )
# collectors_matches_group_simplified.drop_duplicates(inplace=True)
# collectors_matches_group_simplified.head()
# ## cell split - code
# 
# # old file meise_collectors_matches_wikidata_items_group_concat_%s.csv
# this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_wditems_group_concat_wdlabels-joined_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_matches_group.to_csv(this_output_file)
# 
# print("Wrote groups of collectors matches into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
# ## cell split - markdown
# 
# # ### Merge Data to Individual WikiData Items
# # 
# # For this, merge by namematch_resource_data and focus to get individual WikiData items.
# 
# ## cell split - code
# 
# print('Merge simply namematch_resource_data to Wiki data for abbreviated and full names... ')
# for i, wd_matching_column in enumerate(['canonical_string', 'canonical_string_fullname']):
# 
#     # join wikidata items to avh collectors matches
#     #   avh_matches = pd.merge(avh, matches, left_on='label', right_on='name')
#     #   avh_matches_t1 = pd.merge(avh_matches, wikidata, left_on='matched_name', right_on='canonical_string')
#     # link counts of wikidata items with same canonical name string
#     #   avh_matches_t2 = pd.merge(avh_matches_t1, wd_test, left_on="matched_name", right_on="canonical_string")
#     #   avh_matches_t2.rename(columns = {list(avh_matches_t2.columns)[-1]: 'dup_count'}, inplace=True)
#     
#     print('Run %s:   Merge all (collectors matches) using namematch_resource_data, add wikidata ...' % (i + 1))
#     collectors_matches_wd1 = pd.merge(
#         collectors_all_matches, wikidata,
#         left_on='namematch_resource_data', right_on=wd_matching_column,
#         suffixes=('__coll_all_matches', '__wd')
#         # append to left-data, right-data only when identical column names occur
#     )
# 
#     print('Run %s:   Build data frame “collectors_matches_with_wdata” ...' % (i + 1))
#     collectors_matches_with_wdata = collectors_matches_wd1 \
#         if i == 0 \
#         else pd.concat([collectors_matches_with_wdata, collectors_matches_wd1], ignore_index=True)
# 
# print('Done')
# ## cell split - code
# 
# pprint.pprint(collectors_matches_with_wdata.columns)
# # echo "${text}" | fold --spaces | sed 's@^@#  @'
# ## cell split - code
# 
# collectors_matches_with_wdata.drop_duplicates(inplace=True)
# display(collectors_matches_with_wdata)
# ## cell split - markdown
# 
# # Save all columns for further analysis
# 
# ## cell split - code
# 
# # old meise_collectors_matches_wikidata-botanists_all-columns_%s.csv
# 
# this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_matches_with_wdata.sort_values(
#     by=['namematch_distance', 'canonical_string_collector_parsed']
#     , inplace=True
# )
# collectors_matches_with_wdata.to_csv(
#     this_output_file, index=False # drop index column
# )
# 
# print("Wrote isolated WikiData items of collector matches into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )
# ## cell split - code
# 
# # TODO meaningful?
# # remove redundant (duplicate (?or empty?)) columns that in any kind are duplicate data (i.e. that we usually do not need)
# # do it by transposing it (https://www.statology.org/pandas-drop-duplicate-columns/)
# compact_df_tmp=collectors_matches_with_wdata.transpose().drop_duplicates().transpose()
# compact_df_tmp.sort_values(
#     by=['namematch_distance', 'canonical_string_collector_parsed']
#     , inplace=True
# )
# 
# # old meise_collectors_matches_wikidata-botanists_all-columns-made-unique_%s.csv
# # results_meise_collectors_vs_wikidata-botanists_kneighbor_names-atomized_all-columns_%s.csv
# this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_names-atomized_all-columns-compact_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# compact_df_tmp.to_csv(
#     this_output_file, index=False # drop index column
# )
# 
# print("Wrote isolated WikiData items (unique columns) of collector matches into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

## Output Mapping to DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

## Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [23]:
# TODO review further evaluation or filtering, counting, clean up aso.
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_distance', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod', 'wyb',
       'wye', 'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='object')


In [24]:
# refactor namematch_similarity → namematch_distance
# refactor namematch_similarity_annotation → namematch_distance_annotation
# refactor custom_namematch_similarity → custom_namematch_namematch
# refactor sort_values
collectors_wikidata_kmeans = collectors_matches_g1_merged_wikidata[
    ['canonical_string_collector_parsed', 'family', 'given', 
     'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
     'source_data',
    'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
    'item', 'canonical_string', 'itemLabel',
    'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
    'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
     'yob', 'yod', 'wyb'
    ]
]

# Order by similarity (desc), number of Wikidata items (asc) and number of collections (desc)
collectors_wikidata_kmeans.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)

dwcagent_attr_output=collectors_wikidata_kmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given', # given TODO übertragen, und richtig sortieren: family, given, alle 4 neuen Berechnungen nochmals durchrechnen
    "namematch_distance", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy()

dwcagent_attr_output['canonical_string_collector_parsed'].replace(to_replace=r'([^,]+),\s*(.+)', value='\\2 \\1', inplace=True, regex=True)
dwcagent_attr_output['namematch_distance_annotation'] = dwcagent_attr_output['namematch_distance'].astype(str).str.replace(r'(.+)', '\\1 (k-means distance)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_distance_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime"] = 0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_annotation', '', allow_duplicates=True)

# df.loc[(df['column_of_interest'] … condition), 'fill_to_column'] = value 

dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
# True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"],
    "custom_score_lifetime"
] = 1
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(),
    "custom_score_lifetime"
] = 0

# False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -1
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = 0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True),
    "custom_score_lifetime"
] = 0.5

# False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()),
    "custom_score_lifetime"
] = -0.5
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False),
    "custom_score_lifetime"
] = -0.5

# annotations True cases
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "full match"

# annotations True cases but <NA> missing values
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"] & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "OK? year of death is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"], 
    "custom_score_lifetime_annotation"
] = "OK? year of birth is missing"
dwcagent_attr_output.loc[
    dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull() & dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull(), 
    "custom_score_lifetime_annotation"
] = "unknown life time"

# annotations False cases
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "life time not matching any enventDate (yob + %s … yod)" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==True) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == False), 
    "custom_score_lifetime_annotation"
] = "OK yob + %s, but yod not matching, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"] == True), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, OK yod, check name and liftime data" % years_from_birth_until_first_collection_activity
# annotations False cases but <NA> missing values
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"]==False) & (dwcagent_attr_output["yod_is_gt_eventDate_max"].isnull()), 
    "custom_score_lifetime_annotation"
] = "yob + %s not matching, yod unknown, check name and liftime data" % years_from_birth_until_first_collection_activity
dwcagent_attr_output.loc[
    (dwcagent_attr_output["yob_is_lt_eventDate_min"].isnull()) & (dwcagent_attr_output["yod_is_gt_eventDate_max"]==False), 
    "custom_score_lifetime_annotation"
] = "yob unknown, yod not matching, check name and liftime data"

dwcagent_attr_output["custom_score_multiple_names"]=0 # 0 shall mean: we don’t know yet for real
dwcagent_attr_output.loc[
    (dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)),
    'custom_score_multiple_names'
] = -0.5 # one decision has to be made, so cut the range of -1 to 0 only into half (or include multiple count somehow?)

namematch_distance_max=dwcagent_attr_output['namematch_distance'].max()
dwcagent_attr_output['custom_score_overall'] = (
    # reconsider/transform distance (0 … xx, range larger than 1) to similarity (1 … 0, range of 1) for scoring
    abs( dwcagent_attr_output['namematch_distance'] - namematch_distance_max ) / namematch_distance_max * \
    (
        ( dwcagent_attr_output["custom_score_lifetime"] + dwcagent_attr_output['custom_score_multiple_names']) / 2
    )
).round(3)

dwcagent_attr_output['attributionRemarks'] = dwcagent_attr_output.apply(
    lambda row: "{similarity_distance_note};"
                " {score_overall:.2f} (score overall);"
                " {lifetime_periode} (life time);"
                " {lifetime_score:.1f} (life time score);"
                " {lifetime_score_annote} (life time score note);"
                " {score_multinames:.2f} (score multiple names);"
        .format(
    similarity_distance_note=row['namematch_distance_annotation'],
    lifetime_periode=row["life_time_periode"],
    lifetime_score=row["custom_score_lifetime"],
    lifetime_score_annote=row["custom_score_lifetime_annotation"],
    score_overall=row["custom_score_overall"],
    score_multinames=row["custom_score_multiple_names"]
    ), axis='columns'
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    by=['namematch_distance', 'family', 'given', 'custom_score_overall'], 
    ascending=[True, True, True, False], inplace=True
)

# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum()  + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(
    dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 
    'displayOrder', 
    temp_insert_value, 
    allow_duplicates=True
)

# test an show example data
show_display_output=True
if show_display_output:
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime', 'custom_score_lifetime_annotation'
    ]).head(5))

/tmp/ipykernel_23401/252471951.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collectors_wikidata_kmeans.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
765,M. Aalto,Marjatta Aalto,0.5,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1939-?,1975-07-07,1976-09-09,True,<NA>,1.0,OK? year of death is missing
858,E.C. Abbe,Ernst Cleveland Abbe,0.5,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1905-2000,1932-09-01,1932-09-01,True,True,1.0,full match
868,A.T.D. Abbott,A. T. D. Abbott,0.5,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1936-2013,2010-01-07,2010-01-07,True,True,1.0,full match
873,W.L. Abbott,William Louis Abbott,0.5,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1860-1936,1922-02-26,1922-02-26,True,True,1.0,full match
928,J.E.C. Aberdeen,John Errol Chandos Aberdeen,0.5,0.0 (k-means distance); 0.50 (score overall); ...,0.0,0.0,1913-1996,1961-05-04,1964-03-31,True,True,1.0,full match


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime,custom_score_lifetime_annotation
1545,B.R. Adams,Bryan Roger Adams,-0.25,0.0 (k-means distance); -0.25 (score overall);...,0.0,0.0,1942-?,1872-01-01,1979-05-12,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
1826,A. Afzelius,Adam Afzelius,0.25,0.0 (k-means distance); 0.25 (score overall); ...,0.0,0.0,1750-1837,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
1830,P.C. Afzelius,Pehr Conrad Afzelius,0.00,0.0 (k-means distance); 0.00 (score overall); ...,-0.5,0.0,1817-1850,NaT,NaT,False,True,0.5,"yob + 10 not matching, OK yod, check name and ..."
1961,R. Aguilar,Reinaldo Aguilar,-0.25,0.0 (k-means distance); -0.25 (score overall);...,0.0,0.0,1962-?,1945-01-11,2003-11-19,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
2036,S. Ahmad,Shamsuddin Ahmad,-0.50,0.0 (k-means distance); -0.50 (score overall);...,-0.5,0.0,1942-?,1948-08-01,1976-08-18,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."


In [25]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_distance':                 'custom_namematch_distance'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

show_display_output=False
if show_display_output:
    dwcagent_attr_output.head(20)

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_distance',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]


In [26]:
show_display_output=True
if show_display_output:
    criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda x: x < 0 )
    
    display(dwcagent_attr_output[criterion].head(20))

,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,agentType,action,agentIdentifierType,identifier,custom_score_overall,custom_namematch_distance,custom_score_multiple_names,custom_score_lifetime
853,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,A. Abbas,1,Alia Abbas,0.0 (k-means distance); -0.25 (score overall);...,1869-05-04,2001-08-10,Person,collected,wikidata,http://www.wikidata.org/entity/Q60141229,-0.25,0.0,-0.5,0.0
854,http://www.botanicalcollections.be/specimen/BR...,A. Abbas,A. Abbas,2,Abdulla Abbas,0.0 (k-means distance); -0.25 (score overall);...,1869-05-04,2001-08-10,Person,collected,wikidata,http://www.wikidata.org/entity/Q88804360,-0.25,0.0,-0.5,0.0
1420,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,1,Markus Ackermann,0.0 (k-means distance); 0.25 (score overall); ...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504506,0.25,0.0,-0.5,1.0
1421,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,2,Marianne Ackermann,0.0 (k-means distance); -0.25 (score overall);...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q36619087,-0.25,0.0,-0.5,0.0
1422,http://www.botanicalcollections.be/specimen/BR...,M. Ackermann & Ben Böttcher,M. Ackermann,3,Manfred Ackermann,0.0 (k-means distance); -0.25 (score overall);...,2003-06-20,2003-06-20,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112660,-0.25,0.0,-0.5,0.0
1612,http://www.botanicalcollections.be/specimen/BR...,Laurence O. and M.K. Adhikari,M.K. Adhikari,1,Mahesh Kumar Adhikari,0.0 (k-means distance); -0.25 (score overall);...,2001-05-01,2001-05-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q10323105,-0.25,0.0,-0.5,0.0
1613,http://www.botanicalcollections.be/specimen/BR...,Laurence O. and M.K. Adhikari,M.K. Adhikari,2,Mahesh Kumar Adhikari,0.0 (k-means distance); -0.25 (score overall);...,2001-05-01,2001-05-01,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112706,-0.25,0.0,-0.5,0.0
1830,http://www.botanicalcollections.be/specimen/BR...,P.C. Afzelius,P.C. Afzelius,1,Pehr Conrad Afzelius,0.0 (k-means distance); 0.00 (score overall); ...,NaT,NaT,Person,collected,wikidata,http://www.wikidata.org/entity/Q21504600,0.00,0.0,-0.5,0.5
1829,http://www.botanicalcollections.be/specimen/BR...,P.C. Afzelius,P.C. Afzelius,2,Pehr Conrad Afzelius,0.0 (k-means distance); -0.25 (score overall);...,NaT,NaT,Person,collected,wikidata,http://www.wikidata.org/entity/Q10347935,-0.25,0.0,-0.5,0.0
2037,http://www.botanicalcollections.be/specimen/BR...,Ahmad S.,S. Ahmad,1,Sultan Ahmad,0.0 (k-means distance); 0.25 (score overall); ...,1948-08-01,1976-08-18,Person,collected,wikidata,http://www.wikidata.org/entity/Q47112765,0.25,0.0,-0.5,1.0


In [27]:
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
this_timestamp_for_data=20230913
this_output_file='data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_meise_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_20230913.csv (27025 kB)


## Documentation

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest